In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/HW1_NNF/')  # Add the directory containing the file to the Python path
import SeprationIndex  # Import your module

Importing essential libraries and defining flatten transformation

In [ ]:
import torch
import torchvision
from torchvision import transforms
from SeprationIndex import Kalhor_SeparationIndex

# Define a custom transformation to flatten the images
class FlattenTransform:
    def __call__(self, image):
        return image.view(-1)

# Define the transformations
transform = transforms.Compose([transforms.ToTensor(), FlattenTransform()])


spiliting training dataset \\
finding instance for its data \\
calculating SI

In [ ]:
from torch.utils.data import ConcatDataset
#Download train dataset from mnist dataset
dataset  = torchvision.datasets.MNIST(root='./data',train=True,transform=transform,download=True)


num_samples_per_class = 1000


total_samples = num_samples_per_class * 10


class_indices = dataset.targets.unique()
datasets_per_class = []
for class_index in class_indices:
    class_samples = [i for i, label in enumerate(dataset.targets) if label == class_index]
    selected_samples = class_samples[:num_samples_per_class]
    selected_dataset = torch.utils.data.Subset(dataset, selected_samples)
    datasets_per_class.append(selected_dataset)

dataset = ConcatDataset(datasets_per_class)

# Get the data and labels from the concatenated dataset
data_concatenated = torch.stack([dataset[i][0] for i in range(len(dataset))])
labels_concatenated = torch.tensor([dataset[i][1] for i in range(len(dataset))]).view(-1, 1)

# Create an instance of Kalhor_SeparationIndex
instance_concatenated = Kalhor_SeparationIndex(data_concatenated, labels_concatenated)

# Compute the Separation Index for the concatenated dataset
si_concatenated = instance_concatenated.si()

# Print the Separation Index for the concatenated dataset
print("Separation Index for the Concatenated Dataset:", si_concatenated.detach().cpu().numpy())

100%|██████████| 9912422/9912422 [00:00<00:00, 86378165.23it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 11537831.59it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 28376496.69it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3664972.83it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Separation Index for the Concatenated Dataset: 0.952


In [ ]:
# Calculate high-order SI with order 2
order = 2
high_si = instance_concatenated.high_order_si(order)
print('High-order Separation Index (Order {}):'.format(order), high_si.detach().cpu().numpy())

# Calculate high-order soft SI with order 3
order = 3
soft_si = instance_concatenated.soft_order_si(order)
print('High-order Soft Separation Index (Order {}):'.format(order), soft_si.detach().cpu().numpy())

# Calculate center-based SI
csi = instance_concatenated.center_si()
print('Center-based Separation Index:', csi.detach().cpu().numpy())

# Calculate anti-SI with order 2
order = 2
anti_si = instance_concatenated.anti_si(order)
print('Anti Separation Index (Order {}):'.format(order), anti_si.detach().cpu().numpy())


High-order Separation Index (Order 2): 0.9136
High-order Soft Separation Index (Order 3): 0.9361333
Center-based Separation Index: 0.8038
Anti Separation Index (Order 2): 0.0263


Calculating SI for test dataset

In [ ]:
# Load the MNIST test dataset
mnist_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Get the data and labels from the test set
data = torch.stack([mnist_dataset[i][0] for i in range(len(mnist_dataset))])
labels = torch.tensor([mnist_dataset[i][1] for i in range(len(mnist_dataset))])
labels = labels.view(-1, 1)

# Create an instance of Kalhor_SeparationIndex
instance = Kalhor_SeparationIndex(data, labels)

# Compute the Separation Index
si = instance.si()

# Print the Separation Index
print("Separation Index:", si.detach().cpu().numpy())


Separation Index: 0.9558


In [ ]:
# Calculate high-order SI with order 2
order = 2
high_si = instance.high_order_si(order)
print('High-order Separation Index (Order {}):'.format(order), high_si.detach().cpu().numpy())

# Calculate high-order soft SI with order 3
order = 3
soft_si = instance.soft_order_si(order)
print('High-order Soft Separation Index (Order {}):'.format(order), soft_si.detach().cpu().numpy())

# Calculate center-based SI
csi = instance.center_si()
print('Center-based Separation Index:', csi.detach().cpu().numpy())

# Calculate anti-SI with order 2
order = 2
anti_si = instance.anti_si(order)
print('Anti Separation Index (Order {}):'.format(order), anti_si.detach().cpu().numpy())


High-order Separation Index (Order 2): 0.9188
High-order Soft Separation Index (Order 3): 0.9410333
Center-based Separation Index: 0.8229
Anti Separation Index (Order 2): 0.0235


----------------------------------بخش ج---------------------------------\


In [ ]:
import torch.nn as nn
class AlexNet(nn.Module):
    def __init__(self, num=10):
        super(AlexNet, self).__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d( kernel_size=2, stride=2),
            nn.Conv2d(64, 96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(96, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d( kernel_size=2, stride=1),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(32*12*12,2048),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(2048,1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024,num),

        )

    def forward(self, x):

        x = self.feature(x)
        x = x.view(-1,32*12*12)
        x = self.classifier(x)
        return x

reading the saved weight and using them on alexnet \
spiliting 500 data from each classes: 5000 data from training dataset \

In [ ]:
import torch
import torchvision
from torchvision import transforms
from SeprationIndex import Kalhor_SeparationIndex
# Define a custom transformation to flatten the images and normalize them (if required):
class FlattenTransform:
    def __call__(self, image):
        return image.view(-1)

# Define the transformations
transform = transforms.Compose([transforms.ToTensor()])

from torch.utils.data import ConcatDataset
#Download train dataset from mnist dataset
dataset  = torchvision.datasets.MNIST(root='./data',train=True,transform=transform,download=True)


num_samples_per_class = 500


total_samples = num_samples_per_class * 10


class_indices = dataset.targets.unique()
datasets_per_class = []
for class_index in class_indices:
    class_samples = [i for i, label in enumerate(dataset.targets) if label == class_index]
    selected_samples = class_samples[:num_samples_per_class]
    selected_dataset = torch.utils.data.Subset(dataset, selected_samples)
    datasets_per_class.append(selected_dataset)

dataset = ConcatDataset(datasets_per_class)

# Get the data and labels from the concatenated dataset
data = torch.stack([dataset[i][0] for i in range(len(dataset))])
labels = torch.tensor([dataset[i][1] for i in range(len(dataset))]).view(-1, 1)
labels = labels.view(-1, 1)  # Ensure the labels have the correct shape

# Load your trained model checkpoint
model_checkpoint = torch.load("/content/drive/MyDrive/HW1_NNF/saved_model.pth")

# Create the model architecture
# Replace 'YourModelClass' with the actual class used to define your model
model = AlexNet()
model.load_state_dict(model_checkpoint)
model.eval()  # Set the model to evaluation mode


AlexNet(
  (feature): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(96, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=4608, out_features=2048, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): ReLU(in

extracting last layer's features

In [ ]:
# Create a forward hook function to capture features before the fully connected layers
features = None

def hook(module, input, output):
    global features
    features = output

# Create a forward hook function to capture features before the fully connected layers
features = None

def hook(module, input, output):
    global features
    features = output

# Register the hook to the last layer
model.feature[-1].register_forward_hook(hook)

# Pass data through the model
with torch.no_grad():
    output = model(data)
features = features.view(features.size(0), -1)

finding Intance and SI...

In [ ]:
# Create an instance of Kalhor_SeparationIndex
instance2 = Kalhor_SeparationIndex(features, labels)

# Compute the Separation Index
si = instance2.si()

# Print the Separation Index
print("Separation Index:", si.detach().cpu().numpy())


Separation Index: 0.9794


In [ ]:
# Create an instance of Kalhor_SeparationIndex
instance2 = Kalhor_SeparationIndex(features, labels)

# Compute the Separation Index
si = instance2.si()

# Print the Separation Index
print("Separation Index:", si.detach().cpu().numpy())


Separation Index: 0.9794


In [ ]:
# Calculate high-order SI with order 2
order = 2
high_si = instance2.high_order_si(order)
print('High-order Separation Index (Order {}):'.format(order), high_si.detach().cpu().numpy())

# Calculate high-order soft SI with order 3
order = 3
soft_si = instance2.soft_order_si(order)
print('High-order Soft Separation Index (Order {}):'.format(order), soft_si.detach().cpu().numpy())



High-order Separation Index (Order 2): 0.9642
High-order Soft Separation Index (Order 3): 0.9746


In [ ]:
# Calculate center-based SI
csi = instance2.center_si()
print('Center-based Separation Index:', csi.detach().cpu().numpy())

# Calculate anti-SI with order 2
order = 2
anti_si = instance2.anti_si(order)
print('Anti Separation Index (Order {}):'.format(order), anti_si.detach().cpu().numpy())


Center-based Separation Index: 0.9518
Anti Separation Index (Order 2): 0.0106


----------------------------------بخش د-----------------------------------\
Finding Cross SI

In [ ]:
# Load the MNIST test dataset
mnist_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Get the data and labels from the test set
data = torch.stack([mnist_dataset[i][0] for i in range(len(mnist_dataset))])
labels = torch.tensor([mnist_dataset[i][1] for i in range(len(mnist_dataset))])
labels = labels.view(-1, 1)

# Calculate cross separation index for classes
cross_si_class = instance_concatenated.cross_si_class(data, labels)
print("Cross Separation Index for Classes:", cross_si_class.detach().cpu().numpy())

# Calculate cross separation index for data points
cr_si_data = instance_concatenated.cross_si(data, labels)
print("Cross Separation Index:", cr_si_data.detach().cpu().numpy())


Cross Separation Index for Classes: [0.99081635 0.99559474 0.9302326  0.9306931  0.9327902  0.9450673
 0.9791232  0.9357977  0.8880904  0.9326065 ]
Cross Separation Index: 0.9466


--------------------بخش و-------------------\

In [ ]:
import torch
from sklearn.datasets import fetch_california_housing
from SmoothnessIndex import Kalhor_SmoothnessIndex

# Load California housing dataset
california_housing = fetch_california_housing()
data = torch.tensor(california_housing.data, dtype=torch.float32, device=device)
target = torch.tensor(california_housing.target.reshape(-1, 1), dtype=torch.float32, device=device)

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Create an instance of Kalhor_SmoothnessIndex
housing_instance = Kalhor_SmoothnessIndex(data, target)

# Calculate the Smoothness Index
smi_linear = housing_instance.smi_linear()
print('Linear Smoothness Index for the California housing dataset is:', smi_linear.detach().cpu().numpy())

Linear Smoothness Index for the California housing dataset is: 0.7351261


In [ ]:
import torch
from sklearn.datasets import fetch_california_housing
from SmoothnessIndex import Kalhor_SmoothnessIndex
from sklearn.model_selection import train_test_split

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load California housing dataset
california_housing = fetch_california_housing()

# Convert to PyTorch tensors
data = torch.tensor(california_housing.data, dtype=torch.float32, device=device)
target = torch.tensor(california_housing.target.reshape(-1, 1), dtype=torch.float32, device=device)

# Split the data into training and testing sets (80% train, 20% test)
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.2, random_state=42)

# Create an instance of Kalhor_SmoothnessIndex
housing_instance = Kalhor_SmoothnessIndex(data_train, target_train)

# Calculate the Cross Smoothness Index
cross_smi_linear = housing_instance.cross_smi_linear(data_test, target_test)
print('Cross Smoothness Index for the California housing dataset is:', cross_smi_linear.detach().cpu().numpy())


Cross Smoothness Index for the California housing dataset is: 0.7328244
